In [1]:
import pandas as pd
import numpy as np
import keras

Using TensorFlow backend.


In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Lambda, BatchNormalization, Flatten
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

In [3]:
X_train = np.load('../data/X_train.npy').reshape(36000,28,28,1)
X_valid = np.load('../data/X_valid.npy').reshape(6000,28,28,1)
y_train = np.load('../data/y_train.npy')
y_valid = np.load('../data/y_valid.npy')
print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)

(36000, 28, 28, 1) (36000, 10)
(6000, 28, 28, 1) (6000, 10)


### Data Augmentation

In [4]:
transform = {}
transform['width_shift_range'] = 0.075
transform['height_shift_range'] = 0.075
transform['rotation_range'] = 5
transform['shear_range'] = 0.3
transform['zoom_range'] = 0.075

generator = ImageDataGenerator(**transform)
generator.fit(X_train)

/home/ubuntu/.software/anaonda3/lib/python3.6/site-packages/keras/preprocessing/image.py:648: UserWarning: Expected input to be images (as Numpy array) following the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (36000, 28, 28, 1) (1 channels).
  ' (' + str(x.shape[self.channel_axis]) + ' channels).')


### Model Architecture

In [5]:
model = Sequential()
model.add(BatchNormalization(input_shape=(28,28,1), axis=-1))
model.add(Conv2D(32, (3,3), strides=(1,1), padding='same', activation='relu'))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(32, (3,3), strides=(1,1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(64, (3,3), strides=(1,1), padding='same', activation='relu'))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(64, (3,3), strides=(1,1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [6]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 28, 28, 1)         4         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
batch_normalization_2 (Batch (None, 28, 28, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 14, 14, 32)        128       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 64)        18496     
__________

In [8]:
params = {}
params['epochs'] = 1
params['steps_per_epoch'] = 1200 # 36000 / 30
params['validation_data'] = (X_valid, y_valid)

### Train Model

In [9]:
model.optimizer.lr = 0.001
params['epochs'] = 1
model.fit_generator(generator.flow(X_train, y_train, batch_size=30), **params)

Epoch 1/1
1200/1200 [==============================] - 93s - loss: 0.2674 - acc: 0.9199 - val_loss: 0.0746 - val_acc: 0.9775


In [10]:
model.optimizer.lr = 0.1
params['epochs'] = 1
model.fit_generator(generator.flow(X_train, y_train, batch_size=30), **params)

Epoch 1/1
1200/1200 [==============================] - 91s - loss: 0.1235 - acc: 0.9626 - val_loss: 0.0489 - val_acc: 0.9867


In [11]:
model.optimizer.lr = 0.01
params['epochs'] = 3
model.fit_generator(generator.flow(X_train, y_train, batch_size=30), **params)

Epoch 1/3
1200/1200 [==============================] - 91s - loss: 0.1024 - acc: 0.9689 - val_loss: 0.0435 - val_acc: 0.9885
Epoch 2/3
1200/1200 [==============================] - 91s - loss: 0.0878 - acc: 0.9726 - val_loss: 0.0397 - val_acc: 0.9887
Epoch 3/3
1200/1200 [==============================] - 91s - loss: 0.0827 - acc: 0.9745 - val_loss: 0.0489 - val_acc: 0.9883


In [12]:
model.optimizer.lr = 0.005
params['epochs'] = 2
model.fit_generator(generator.flow(X_train, y_train, batch_size=30), **params)

Epoch 1/2
1200/1200 [==============================] - 91s - loss: 0.0767 - acc: 0.9762 - val_loss: 0.0288 - val_acc: 0.9935
Epoch 2/2
1200/1200 [==============================] - 91s - loss: 0.0709 - acc: 0.9786 - val_loss: 0.0345 - val_acc: 0.9920


In [13]:
model.save_weights('../model/model_2.h5')

### Score Model

In [14]:
X_score = np.load('../data/X_score.npy').reshape(28000,28,28,1)

In [15]:
predict = model.predict_classes(X_score)

28000/28000 [==============================] - 21s    


In [16]:
predict = pd.DataFrame(predict, columns=['Label'])
predict = predict.reset_index()
predict.columns = ['ImageId','Label']
predict['ImageId'] = predict['ImageId'] + 1

In [17]:
predict.to_csv('../data/submit_v2.csv', index=False)

In [ ]:
### Submission scored 0.99014 on LB